In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, Dataset, DatasetDict

2024-04-19 12:14:46.310444: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 12:14:46.364273: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-19 12:14:47.709181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1
CUDA version: 11.8
cuDNN version: 8700
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.37.2
Datasets version: 2.14.5


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Fri Apr 19 12:14:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        On  | 00000000:41:00.0 Off |                  N/A |
| 32%   55C    P2             216W / 350W |    941MiB / 24576MiB |     41%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache() 

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 41% |  4% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  0% |  0% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 41% |  4% |
|  1 |  0% |  0% |
|  2 |  0% |  0% |
|  3 |  5% |  1% |


In [9]:
data = pd.read_csv("Final_cleaned_test_types.csv" , index_col = 0)
data

label
text_clean                                               
omnilight and spotlight incorrect on x11 editor...      1
failing test jest integration testssrcpluginsfi...      1
flaky testtendermintbuffer subject of the issue...      1
unable to read logs from sway testswhen running...      1
failing test execution context functional tests...      1
...                                                   ...
interface icons inconsistent security issues pl...      2
create propactiveintegrationtests module backgr...      2
bug test client oom for client config clientran...      2
request for ability to orderfilter test suites ...      2
release 438 release candidate 1 ruleset test ma...      2

[2982 rows x 1 columns]

In [10]:
# Smaller and faster than bert.
base_model_id = "distilbert-base-uncased"

epochs = 5 #Number of full cyles through the training set.
num_labels = 2 
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evalutaion examples in on iteratoion.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

In [11]:
# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [12]:
# Drops rows with missing values
data.dropna(inplace=True)

In [13]:
# Resets the index after dropping rows
data.reset_index(inplace=True)
data

,text_clean,label
0,omnilight and spotlight incorrect on x11 edito...,1
1,failing test jest integration testssrcpluginsf...,1
2,flaky testtendermintbuffer subject of the issu...,1
3,unable to read logs from sway testswhen runnin...,1
4,failing test execution context functional test...,1
...,...,...
2977,interface icons inconsistent security issues p...,2
2978,create propactiveintegrationtests module backg...,2
2979,bug test client oom for client config clientra...,2
2980,request for ability to orderfilter test suites...,2


In [15]:
data


,text_clean,label
0,omnilight and spotlight incorrect on x11 edito...,1
1,failing test jest integration testssrcpluginsf...,1
2,flaky testtendermintbuffer subject of the issu...,1
3,unable to read logs from sway testswhen runnin...,1
4,failing test execution context functional test...,1
...,...,...
2977,interface icons inconsistent security issues p...,2
2978,create propactiveintegrationtests module backg...,2
2979,bug test client oom for client config clientra...,2
2980,request for ability to orderfilter test suites...,2


In [16]:
# 80% trainig, 10% validate, 10% test. Seed 42.
# Test 80-10-10 and 70-15-15
train , validate , test = train_validate_test_split(data)


In [17]:
train.set_index("label" , inplace = True)
validate.set_index("label" , inplace = True)
test.set_index("label" , inplace = True)

In [18]:
test

,text_clean
label,
5,bugios the spacebar caption doesnt get refresh...
5,backport v11 bug no resource limits set for th...
2,merge tests from vaadinflow 90 configure sprin...
5,add initial e2e testsdescribe the solution you...
2,corevchanxen v419 r42update of corevchanxen to...
...,...
5,write tests for dexwrite tests for the dex mod
5,typescript client test to manage rpc serverfor...
5,additional checks and tests for muteto be cons...


In [20]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
testds = Dataset.from_pandas(test)


ds = DatasetDict()

ds["test"] = testds
ds["train"] = tds
ds["validate"] = vds


ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 299
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 2385
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 298
    })
})

In [21]:
train_dataset = ds["train"]
valid_dataset = ds["validate"]
test_ds = ds["test"]

In [22]:
ds["train"][0]

{'text_clean': 'general sanity checks on gamestest for compatibility with database and frontend game actions should be replayable prevent situations where a deserialized object is not the same object as the one in game causing updates on something else replaying actions should generate the same view as when the original action was made store jsonfiles of replays that are tested with assertions of what is to be expected this can also detect for viewchanges or breaking replaychanges no kotlinspecific data sent to frontend data is correctly serializable and deserializable tofrom database use a random amount of players in dslrandomplaytest with a flexible amount of wsclients if these tests take too long time perhaps look into a way to not run them for each jenkinsbuild',
 'label': 2}

In [23]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=num_labels)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Tokanization

In [25]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [27]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=len(valid_dataset))
test_dataset = test_ds.map(tokenize, batched=True, batch_size=len(test_ds))


Map:   0%|          | 0/2385 [00:00<?, ? examples/s]

Map:   0%|          | 0/298 [00:00<?, ? examples/s]

Map:   0%|          | 0/299 [00:00<?, ? examples/s]

In [28]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [29]:
 trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

In [32]:
export CUDA_LAUNCH_BLOCKING=1
export TORCH_USE_CUDA_DSA=1


SyntaxError: invalid syntax (2127537172.py, line 1)

In [31]:
trainer.train() 

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [30]:
eval_result = trainer.evaluate(eval_dataset=valid_dataset)

In [32]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.8631273330568229

eval_f1 = [0.86169321 0.86453202]

eval_loss = 0.7703143358230591

eval_precision = [0.86751055 0.8588907 ]

eval_recall = [0.85595337 0.87024793]

eval_runtime = 77.027

eval_samples_per_second = 93.902

eval_steps_per_second = 2.947



## Training loss decreases, valdiation loss increases = Overfitting

In [33]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [34]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.8679662657265311

eval_f1 = [0.86428876 0.87144972]

eval_loss = 0.7561773061752319

eval_precision = [0.85686109 0.8786645 ]

eval_recall = [0.87184633 0.86435247]

eval_runtime = 77.085

eval_samples_per_second = 93.832

eval_steps_per_second = 2.945



# Test set

In [35]:
separate_test_set_results = trainer.evaluate(eval_dataset=separate_test_set_dataset)

In [36]:
for key, value in sorted(separate_test_set_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.8667413213885778

eval_f1 = [0.86680761 0.86667497]

eval_loss = 0.7555153369903564

eval_precision = [0.86799502 0.86548981]

eval_recall = [0.86562345 0.86786338]

eval_runtime = 82.7271

eval_samples_per_second = 97.151

eval_steps_per_second = 3.046



In [37]:
trainer.save_model(model_dir + "_local") 

In [45]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local", truncation=True)

In [46]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [47]:
classifier("this contain bugs regarding testing")

[{'label': 'LABEL_1', 'score': 0.8895082473754883}]

In [48]:
classifier("this contain bugs regarding automtion not testing")

[{'label': 'LABEL_0', 'score': 0.9836528301239014}]

In [49]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_1', 'score': 0.9379398822784424}]

In [50]:
import pandas as pd
df = pd.read_csv('clean_test_or_not_test_debt.csv',index_col = 0)
df.head()

,text_clean,label
0,make sure integration tests are running with i...,1
1,cover anvil service response in unit testshttp...,1
2,service apprepositorycontroller should be cach...,1
3,many caches persist between test casesan examp...,1
4,improve friendliness of behat text pattern mat...,1


In [51]:
def classify_text(text):
    # Get prediction
    result = classifier(text)
    # Return the label of the highest scoring classification
    return result[0]['label']

# Apply the classification function to your text column (assuming it's named 'text_clean')
df['predicted_label'] = df['text_clean'].apply(classify_text)


In [52]:
df.head(100)

,text_clean,label,predicted_label
0,make sure integration tests are running with i...,1,LABEL_1
1,cover anvil service response in unit testshttp...,1,LABEL_1
2,service apprepositorycontroller should be cach...,1,LABEL_0
3,many caches persist between test casesan examp...,1,LABEL_1
4,improve friendliness of behat text pattern mat...,1,LABEL_1
...,...,...,...
95,ci test flake can modify upstreams repeatedly ...,1,LABEL_1
96,fix broken shebang on smoke test script contin...,1,LABEL_1
97,add webworkers tests description add unit test...,1,LABEL_1
98,servlet 60 correct the list of valid orderings...,1,LABEL_0


In [55]:
from sklearn.metrics import accuracy_score, f1_score

# Assuming your predicted labels are in a format like 'LABEL_0', 'LABEL_1', etc.
# Convert these to numeric by extracting the number part and converting it to an integer
df['predicted_label_numeric'] = df['predicted_label'].apply(lambda x: int(x.split('_')[1]))

# Now, your actual labels are assumed to be in the 'label' column and already numeric
actual_labels = df['label']

# Predicted labels are now in 'predicted_label_numeric'
predicted_labels = df['predicted_label_numeric']


df.head()

,text_clean,label,predicted_label,predicted_label_numeric
0,make sure integration tests are running with i...,1,LABEL_1,1
1,cover anvil service response in unit testshttp...,1,LABEL_1,1
2,service apprepositorycontroller should be cach...,1,LABEL_0,0
3,many caches persist between test casesan examp...,1,LABEL_1,1
4,improve friendliness of behat text pattern mat...,1,LABEL_1,1


In [57]:
df.to_csv('TestNotTestRes.csv', index=False)

### Delete the dataset with large memory

In [58]:
# Calculate accuracy
accuracy = accuracy_score(actual_labels, predicted_labels)

# Calculate F1 score
# Assuming it's a binary classification, you could specify the average method if it's multi-class
f1 = f1_score(actual_labels, predicted_labels, average='binary' if df['label'].nunique() == 2 else 'weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.8183673469387756
F1 Score: 0.831758034026465


In [59]:
del valid_dataset

In [60]:
del model

In [61]:
# Free cache
torch.cuda.empty_cache()

In [62]:
!nvidia-smi

Wed Mar 13 22:13:42 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  | 00000000:01:00.0 Off |                  N/A |
| 22%   28C    P8              28W / 250W |  10503MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--